### AVANCE 1 - PROYECTO INTEGRADOR 

IMPORTACIÓN DE DATOS CSV

Para realizar la importación de los datos se realizó por fragmentos, ya que al ser un archivo con mucha cantidad de datos, no permitió hacer la carga de otra manera. 

In [2]:
#Importar Librerias

import pandas as pd
import numpy as np

In [3]:
# Ruta al archivo CSV
ruta_archivo = r"C:\Users\ikiga\OneDrive\Escritorio\data_latinoamerica.csv"

# Definir el tamaño del fragmento
chunksize = 100000

# Leer el archivo CSV en fragmentos
df_chunks = pd.read_csv(ruta_archivo, parse_dates=['date'], chunksize=chunksize)

# Recorrer cada fragmento del archivo
for chunk in df_chunks:

# Verificar las primeras filas para confirmar la carga y el tipo de datos
 print(chunk.head())
 print(chunk.dtypes)

  location_key       date country_code country_name  new_confirmed  \
0           AR 2020-01-01           AR    Argentina            3.0   
1           AR 2020-01-02           AR    Argentina           14.0   
2           AR 2020-01-03           AR    Argentina            3.0   
3           AR 2020-01-04           AR    Argentina            7.0   
4           AR 2020-01-05           AR    Argentina            5.0   

   new_deceased  cumulative_confirmed  cumulative_deceased  \
0           0.0                   3.0                  0.0   
1           0.0                  17.0                  0.0   
2           0.0                  20.0                  0.0   
3           0.0                  27.0                  0.0   
4           0.0                  32.0                  0.0   

   cumulative_vaccine_doses_administered  population  ...  \
0                                    NaN  44938712.0  ...   
1                                    NaN  44938712.0  ...   
2                      

VERIFICAR FILAS Y COLUMNAS 

In [ ]:

df_chunks = pd.read_csv(ruta_archivo, parse_dates=['date'], chunksize=100000)

total_filas = 0
columnas = None

# Recorrer los fragmentos
for chunk in df_chunks:
    total_filas += len(chunk)
    if columnas is None:
        columnas = chunk.columns

print("Cantidad total de filas:", total_filas)
print("Cantidad total de columnas:", len(columnas))

# Validación con los valores esperados
if total_filas == 12216057 and len(columnas) == 50:
    print("✅ Las filas y columnas coinciden con lo esperado según la consigna.")
else:
    print("⚠️ Las filas y/o columnas NO coinciden con lo esperado.")




FILTRAR POR PAIS Y FECHA

Se realiza el filtrado por fecha y paises, y se hace el recorrido por fragmentos, y se unifica en un solo DataFrame llamado data_filtrada.

In [ ]:

paises_objetivo = ["Colombia", "Argentina", "Chile", "Mexico", "Peru", "Brazil"]

df_chunks = pd.read_csv(
    ruta_archivo,
    parse_dates=['date'], 
    dtype={'country_name': 'category'},  # Optimiza el uso de memoria
    chunksize=50000,
    on_bad_lines='skip',
    low_memory=True
)

# Lista para acumular fragmentos filtrados
fragmentos_filtrados = []

# Filtrar por país y fecha en cada fragmento
for chunk in df_chunks:
    if 'country_name' in chunk.columns and 'date' in chunk.columns:
        filtrado = chunk.loc[
            (chunk['country_name'].isin(paises_objetivo)) & 
            (chunk['date'] >= "2021-01-01")
        ]
        fragmentos_filtrados.append(filtrado)

# Unir los fragmentos filtrados en un único DataFrame
data_filtrada = pd.concat(fragmentos_filtrados, ignore_index=True)

print(f"""
Filtrado completo ✅
Total de filas luego del filtrado: {len(data_filtrada)}
Total de columnas: {len(data_filtrada.columns)}
""")

OPERAR SOBRE NULOS 

In [ ]:
#Operar sobre filas y columnas totalmente en blanco.
data_nan= data_filtrada.dropna(axis=0, how = "all") # Quitar filas en blanco
data_nan= data_filtrada.dropna(axis=1, how = "all") # Quitar columnas en blanco


In [ ]:
#Obtener el total de valores nulos que contiene cada campo del dataFrame
data_nan.isnull().sum()

COMPARACIÓN A NIVEL PAIS 

In [ ]:
#Identificar el contenido de location key y filtrar solo a nivel país. 
df_final = data_nan[data_nan["location_key"].isin(["AR","CL","CO","MX","PE","BR"])]

VERIFICAR DF_FINAL Y SEGUIR TRABAJANDO CON NULOS 

In [ ]:
df_final.shape

In [ ]:
#Validar columnas con valores nulos 
df_final.isnull().sum()[df_final.isnull().sum()>0]

In [ ]:

# Eliminar columnas con más del 70% de nulos
df_final = df_final.drop(columns=['new_recovered', 'cumulative_recovered'])


RELLENAR VALORES NULOS CON INFORMACIÓN 

In [ ]:
# Rellenar con 0: columnas diarias donde ausencia puede interpretarse como 0
df_final.loc[:, ['new_confirmed', 'new_deceased']] = df_final.loc[:, ['new_confirmed', 'new_deceased']].fillna(0)

# Rellenar acumulados con forward fill (último valor conocido)
df_final.loc[:, ['cumulative_confirmed', 'cumulative_deceased', 'cumulative_vaccine_doses_administered']] = df_final.loc[:, ['cumulative_confirmed', 'cumulative_deceased', 'cumulative_vaccine_doses_administered']].ffill()

# Rellenar con el promedio
df_final.loc[:, 'average_temperature_celsius'] = df_final.loc[:, 'average_temperature_celsius'].fillna(df_final['average_temperature_celsius'].mean())

# Rellenar acumulados con forward fill (último valor conocido)
df_final.loc[:, ['minimum_temperature_celsius', 'maximum_temperature_celsius']] = df_final.loc[:, ['minimum_temperature_celsius', 'maximum_temperature_celsius']].ffill()

# Rellenar humedad relativa con interpolación (variable climática continua)
df_final.loc[:, 'relative_humidity'] = df_final.loc[:, 'relative_humidity'].interpolate(method='linear')

# Rellenar precipitación con 0 (días sin lluvia registrada)
df_final.loc[:, 'rainfall_mm'] = df_final.loc[:, 'rainfall_mm'].fillna(0)

CORRECCION DE TIPO DE DATOS 

In [ ]:
# Convertir valores económicos a enteros
df_final['gdp_usd'] = df_final['gdp_usd'].astype(int)

# Asegurar que las coordenadas geográficas mantengan su precisión decimal
df_final['latitude'] = df_final['latitude'].round(6)
df_final['longitude'] = df_final['longitude'].round(6)

# Redondear temperaturas para mejorar presentación
df_final['average_temperature_celsius'] = df_final['average_temperature_celsius'].round(2)
df_final['minimum_temperature_celsius'] = df_final['minimum_temperature_celsius'].round(2)
df_final['maximum_temperature_celsius'] = df_final['maximum_temperature_celsius'].round(2)


# Redondear valores de precipitación
df_final['rainfall_mm'] = df_final['rainfall_mm'].round(2)

# Redondear humedad relativa
df_final['relative_humidity'] = df_final['relative_humidity'].round(2)

# Convertir población de la ciudad más grande a enteros
df_final['population_largest_city'] = df_final['population_largest_city'].astype(int)

# Asegurar que áreas sean enteros, eliminando decimales innecesarios
df_final['area_rural_sq_km'] = df_final['area_rural_sq_km'].astype(int)
df_final['area_urban_sq_km'] = df_final['area_urban_sq_km'].astype(int)

# Redondear valores de esperanza de vida y tasas de mortalidad
df_final['life_expectancy'] = df_final['life_expectancy'].round(2)
df_final['adult_male_mortality_rate'] = df_final['adult_male_mortality_rate'].round(2)
df_final['adult_female_mortality_rate'] = df_final['adult_female_mortality_rate'].round(2)
df_final['pollution_mortality_rate'] = df_final['pollution_mortality_rate'].round(2)
df_final['comorbidity_mortality_rate'] = df_final['comorbidity_mortality_rate'].round(2)

# Corregir valores negativos en casos nuevos confirmados
df_final['new_confirmed'] = df_final['new_confirmed'].clip(lower=0)

# Convertir población de distintos grupos de edad a enteros
df_final[['population_age_00_09', 'population_age_10_19', 'population_age_20_29', 'population_age_30_39']] = \
    df_final[['population_age_00_09', 'population_age_10_19', 'population_age_20_29', 'population_age_30_39']].astype(int)

# Redondear densidad de población y el índice de desarrollo humano
df_final['population_density'] = df_final['population_density'].round(2)
df_final['human_development_index'] = df_final['human_development_index'].round(3)


CARACTERISTICAS DEL DATASET 

In [ ]:
print(df_final.describe())


GUARDAR ARCHIVO .CSV

In [ ]:
df_final.to_csv("DatosFinalesFiltrado.csv", index=False)


ESTADISTICA DESCRIPTIVA 

In [ ]:
# Diccionario para almacenar estadísticas descriptivas
estadisticas = {}

# Iterar sobre cada columna numérica
for columna in df_final.select_dtypes(include=['number']).columns:
    estadisticas[columna] = {
        'Media': df_final[columna].mean(),
        'Mediana': df_final[columna].median(),
        'Desviación estándar': df_final[columna].std(),
        'Máximo': df_final[columna].max(),
        'Mínimo': df_final[columna].min(),
        'Rango': df_final[columna].max() - df_final[columna].min(),
        'Varianza': df_final[columna].var()
    }

# Convertir el diccionario en un DataFrame para visualizarlo mejor
df_estadisticas = pd.DataFrame(estadisticas).T
print(df_estadisticas)


FUNCION PARA ESTADISTICA DESCRIPTIVA 

In [ ]:
# Definir la función
def calcular_estadisticas(df, columna):
    """
    Calcula la mediana, varianza y rango de una columna numérica en un DataFrame.

    Parámetros:
    df (DataFrame): DataFrame de pandas que contiene los datos.
    columna (str): Nombre de la columna sobre la que calcular estadísticas.

    Retorna:
    dict: Diccionario con los valores calculados.
    """
    if columna in df.columns and pd.api.types.is_numeric_dtype(df[columna]):
        estadisticas = {
            'Mediana': df[columna].median(),
            'Varianza': df[columna].var(),
            'Rango': df[columna].max() - df[columna].min()
        }
        return estadisticas
    else:
        return f"La columna '{columna}' no existe o no es numérica."

# Ejemplo de uso
columna_ejemplo = "gdp_usd"  # Cambiar por cualquier columna numérica válida
resultado = calcular_estadisticas(df_final, columna_ejemplo)
print(resultado)


### AVANCE 2 - PROYECTO INTEGRADOR

In [ ]:
#Importar Librerias

import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Cargar el archivo CSV limpio
df_final = pd.read_csv(r"C:\Users\ikiga\OneDrive\Escritorio\PIDA_M4_PI\DatosFinalesFiltrado.csv")


In [ ]:
print(df_final.columns)

ANALISIS ESTADISTICO 

In [ ]:
# Filtrar solo columnas numéricas
df_numerico = df_final.select_dtypes(include=[np.number])

# Medidas estadísticas
estadisticas_generales = df_numerico.describe()
moda = df_numerico.mode().iloc[0]
rango = df_numerico.max() - df_numerico.min()
varianza = df_numerico.var()
desviacion = df_numerico.std()
correlaciones = df_numerico.corr()

# Mostrar resultados
print("✅ Estadísticas generales:\n", estadisticas_generales)
print("\n✅ Moda:\n", moda)
print("\n✅ Rango:\n", rango)
print("\n✅ Varianza:\n", varianza)
print("\n✅ Desviación estándar:\n", desviacion)
print("\n✅ Correlaciones:\n", correlaciones)



VISUALIZACION DE DATOS 

Total de Casos Confirmados por País 

In [ ]:
plt.figure(figsize=(12, 6))

sns.barplot(data=df_final, x='country_name', y='cumulative_confirmed', palette='viridis')

plt.title('Total de Casos Confirmados por País')
plt.xlabel('País')
plt.ylabel('Casos Confirmados Acumulados')

# Ajustar escala del eje Y para mostrar en millones
plt.ticklabel_format(style='plain', axis='y')  # ← Esto evita notación científica
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

Total de Dosis Administradas por País 

In [ ]:

# Configurar el tema oscuro
sns.set_theme(style="darkgrid")

# Crear gráfico de barras
plt.figure(figsize=(12,6))
ax = sns.barplot(data=df_final, x="country_name", y="cumulative_vaccine_doses_administered", palette="viridis")

# Mejorar la visualización
plt.xticks(rotation=90)
plt.xlabel("País")
plt.ylabel("Dosis de vacunas administradas")
plt.title("Total de dosis administradas por país")

# Mostrar gráfico
plt.show()


Total de muertes por País 

In [ ]:

# Configurar el tema oscuro
sns.set_theme(style="darkgrid")

# Crear gráfico de barras
plt.figure(figsize=(12,6))
ax = sns.barplot(data=df_final, x="country_name", y="cumulative_deceased", palette="viridis")

# Mejorar la visualización
plt.xticks(rotation=90)
plt.xlabel("País")
plt.ylabel("Total de muertes")
plt.title("Total de muertes por país")

# Mostrar gráfico
plt.show()


Tasa de Mortalidad por Sexo y País 

In [ ]:

# Configurar la paleta viridis para dos colores
palette_viridis = sns.color_palette("viridis", n_colors=2)

# Agrupar por país y calcular la media de las tasas
df_mortalidad = df_final.groupby('country_name')[['adult_male_mortality_rate', 'adult_female_mortality_rate']].mean().round(2)

# Graficar con viridis
df_mortalidad.plot(kind='bar', figsize=(10, 6), color=palette_viridis)

plt.title('Comparación de Tasa de Mortalidad Adulta por Sexo y País')
plt.xlabel('País')
plt.ylabel('Tasa de Mortalidad')
plt.xticks(rotation=45)
plt.legend(title='Sexo', labels=['Hombres', 'Mujeres'])
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()



Prevalencia de Diabetes y Tabaquismo por País 

In [ ]:
df_prev = df_final.groupby('country_name')[['diabetes_prevalence', 'smoking_prevalence']].mean().reset_index()
df_prev_melt = df_prev.melt(id_vars='country_name', var_name='Condición', value_name='Porcentaje')

plt.figure(figsize=(10,6))
sns.barplot(data=df_prev_melt, x='country_name', y='Porcentaje', hue='Condición', palette='viridis')
plt.title('Prevalencia de Diabetes y Tabaquismo por País')
plt.xlabel('País')
plt.ylabel('% de población')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


MAPA DE CALOR PARA IDENTIFICAR CORRELACIONES 

In [ ]:
#"Mapa de Calor de Correlaciones entre Indicadores de Salud y COVID-19"

# Seleccionar solo las variables de salud y COVID-19
variables_salud_covid = ['cumulative_confirmed', 'cumulative_deceased', 'cumulative_vaccine_doses_administered',
                         'diabetes_prevalence', 'smoking_prevalence', 'pollution_mortality_rate', 'comorbidity_mortality_rate']

# Filtrar columnas y eliminar valores NaN
df_corr = df_final[variables_salud_covid].dropna().corr()

# Asegurar que la matriz de correlación no tenga valores vacíos
df_corr = df_corr.replace([np.nan, np.inf, -np.inf], 0)

plt.figure(figsize=(10, 6))
sns.heatmap(df_corr, annot=True, cmap="viridis", fmt=".2f", linewidths=0.5)

plt.title("Mapa de Calor de Correlaciones entre Indicadores de Salud y COVID-19")
plt.show()

Distribución de la población por rango de edad

In [ ]:


# Diccionario con nombres más claros para el eje X
nombres_edades = {
    "population_age_00_09": "0-9 años", "population_age_10_19": "10-19 años",
    "population_age_20_29": "20-29 años", "population_age_30_39": "30-39 años",
    "population_age_40_49": "40-49 años", "population_age_50_59": "50-59 años",
    "population_age_60_69": "60-69 años", "population_age_70_79": "70-79 años",
    "population_age_80_and_older": "80+ años"
}

# Crear DataFrame con edades agrupadas
df_edades = df_final[list(nombres_edades.keys())].sum().rename(index=nombres_edades)

# Crear gráfico mejorado
plt.figure(figsize=(10,6))
ax = sns.barplot(x=df_edades.index, y=df_edades.values, palette="viridis")

# Agregar valores sobre las barras
for p in ax.patches:
    ax.annotate(f"{int(p.get_height()):,}", (p.get_x() + p.get_width() / 2, p.get_height()),
                ha="center", va="bottom", fontsize=10, color="white")

# Mejorar etiquetas y título
plt.xlabel("Rango de edad", fontsize=12)
plt.ylabel("Total de población", fontsize=12)
plt.title("Distribución de la población por rango de edad", fontsize=14, fontweight="bold")
plt.xticks(rotation=45)

# Mostrar gráfico mejorado
plt.show()

Relación entre la Temperatura Media contra los  Casos Confirmados 

In [ ]:
plt.figure(figsize=(10,6))
sns.scatterplot(data=df_final, x="average_temperature_celsius", y="cumulative_confirmed", palette="viridis", alpha=0.7)

plt.xlabel("Temperatura Media (°C)")
plt.ylabel("Casos Confirmados Acumulados")
plt.title("Relación entre la Temperatura Media y Casos Confirmados")
plt.grid(True, linestyle="--", alpha=0.5)

plt.show()

Relación entre la Temperatura Media contra las muertes confirmadas 

In [ ]:
plt.figure(figsize=(10,6))
sns.scatterplot(data=df_final, x="average_temperature_celsius", y="cumulative_deceased", palette="viridis", alpha=0.7)

plt.xlabel("Temperatura Media (°C)")
plt.ylabel("Muertes Confirmadas Acumuladas")
plt.title("Relación entre la Temperatura Media y Muertes Confirmadas")
plt.grid(True, linestyle="--", alpha=0.5)

plt.show()


Comportamiento de las Dosis Administradas por País (Valor Medio)

In [ ]:


# Agrupar por país y calcular el promedio de dosis administradas
df_vacunas = df_final.groupby("country_name")["cumulative_vaccine_doses_administered"].mean().round(2)

# Crear gráfico de barras
plt.figure(figsize=(12,6))
sns.barplot(x=df_vacunas.index, y=df_vacunas.values, palette="viridis")

# Mejorar visualización
plt.xticks(rotation=90)
plt.xlabel("País")
plt.ylabel("Dosis Administradas (Promedio)")
plt.title("Comportamiento de las Dosis Administradas por País (Valor Medio)")

# Mostrar gráfico
plt.show()


Evolución de Dosis Administradas por Mes y País

In [ ]:
# Convertir la columna 'date' a formato datetime y extraer el mes y año en el formato "ene-21"
df_final["date"] = pd.to_datetime(df_final["date"])
df_final["month"] = df_final["date"].dt.strftime("%b-%y")  # Formato: "ene-21", "feb-21", etc.

# Agrupar por país y mes para calcular el total de dosis administradas
df_evolucion = df_final.groupby(["country_name", "month"])["cumulative_vaccine_doses_administered"].sum().reset_index()

# Crear el gráfico mejorado con viridis
plt.figure(figsize=(12,6))
sns.lineplot(data=df_evolucion, x="month", y="cumulative_vaccine_doses_administered", hue="country_name", palette="viridis", alpha=0.8, marker="o")

# Mejorar visualización
plt.xticks(rotation=45)
plt.xlabel("Mes", fontsize=12)
plt.ylabel("Dosis Administradas", fontsize=12)
plt.title("Evolución de Dosis Administradas por Mes y País", fontsize=14, fontweight="bold")
plt.legend(title="País", loc="upper left", bbox_to_anchor=(1,1))
plt.grid(True, linestyle="--", alpha=0.5)

# Mostrar gráfico
plt.show()

Evolución de Muertes por Mes y País

In [ ]:
# Convertir fecha y extraer mes-año
df_final["date"] = pd.to_datetime(df_final["date"])
df_final["month"] = df_final["date"].dt.strftime("%b-%y")  # Formato "ene-21", "feb-21", etc.

# Agrupar por país y mes para calcular el total de muertes
df_muertes = df_final.groupby(["country_name", "month"])["cumulative_deceased"].sum().reset_index()

# Gráfico de línea
plt.figure(figsize=(12,6))
sns.lineplot(data=df_muertes, x="month", y="cumulative_deceased", hue="country_name", palette="viridis", alpha=0.8, marker="o")

# Mejorar visualización
plt.xticks(rotation=45)
plt.xlabel("Mes")
plt.ylabel("Total de Muertes")
plt.title("Evolución de Muertes por Mes y País")
plt.legend(title="País", loc="upper left", bbox_to_anchor=(1,1))
plt.grid(True, linestyle="--", alpha=0.5)

plt.show()


Casos confirmados por mes de cada país

In [ ]:
# Agrupar por país y mes para calcular el total de casos confirmados
df_casos = df_final.groupby(["country_name", "month"])["cumulative_confirmed"].sum().reset_index()

# Gráfico de línea
plt.figure(figsize=(12,6))
sns.lineplot(data=df_casos, x="month", y="cumulative_confirmed", hue="country_name", palette="viridis", alpha=0.8, marker="o")

# Mejorar visualización
plt.xticks(rotation=45)
plt.xlabel("Mes")
plt.ylabel("Total de Casos Confirmados")
plt.title("Evolución de Casos Confirmados por Mes y País")
plt.legend(title="País", loc="upper left", bbox_to_anchor=(1,1))
plt.grid(True, linestyle="--", alpha=0.5)

plt.show()


Dosis Acumuladas por País

In [ ]:
# Agrupar por país y calcular el total de dosis administradas
df_vacunas = df_final.groupby("country_name")["cumulative_vaccine_doses_administered"].sum().reset_index()

# Crear gráfico de barras con paleta viridis
plt.figure(figsize=(12,6))
sns.barplot(x="country_name", y="cumulative_vaccine_doses_administered", data=df_vacunas, palette="viridis")

# Mejorar visualización
plt.xticks(rotation=90)
plt.xlabel("País")
plt.ylabel("Dosis Administradas (Acumuladas)")
plt.title("Dosis Acumuladas por País")

plt.show()


Distribución de la Temperatura Media por País

In [ ]:

# Crear el boxplot optimizado
plt.figure(figsize=(12,6))
sns.boxplot(
    data=df_final, x="country_name", y="average_temperature_celsius",
    palette="viridis", flierprops={'marker':'o', 'markerfacecolor':'red', 'markersize':5}
)

# Mejorar la visualización
plt.xticks(rotation=45, ha="right")  # Ajusta la orientación de los nombres de países
plt.xlabel("País", fontsize=13)
plt.ylabel("Temperatura Media (°C)", fontsize=13)
plt.title("Distribución de la Temperatura Media por País", fontsize=15, fontweight="bold")
plt.grid(True, linestyle="--", alpha=0.3)  # Mantén la cuadrícula pero hazla más sutil
plt.tight_layout()  # Ajusta el gráfico para evitar solapamientos

# Mostrar gráfico
plt.show()



INCLUIMOS AHORA ALGUNOS GRAFICOS, DONDE ANALIZAMOS NUESTROS DATOS POR MILLON DE HABITANTES, ASI PODEMOS HACER UNA COMPARACIÓN MAS JUSTA

Muertes promedio por millón

In [ ]:

# Asegurarse de que las columnas estén en formato numérico
df_final["cumulative_deceased"] = pd.to_numeric(df_final["cumulative_deceased"], errors="coerce")
df_final["population"] = pd.to_numeric(df_final["population"], errors="coerce")

# Calcular muertes por millón (por país)
df_final["deaths_per_million"] = (df_final["cumulative_deceased"] / df_final["population"]) * 1_000_000

# Agrupar por país y calcular el promedio (por si hay varias fechas)
df_grouped = df_final.groupby("country_name")["deaths_per_million"].mean().reset_index()

# Crear gráfico de barras
plt.figure(figsize=(10,5))
sns.barplot(data=df_grouped, x="country_name", y="deaths_per_million", palette="viridis")

# Mejorar visualización
plt.xlabel("País", fontsize=13)
plt.ylabel("Muertes promedio por millón", fontsize=13)
plt.title("Impacto de la Pandemia: Muertes por Millón de Habitantes por País", fontsize=15, fontweight="bold")
plt.grid(axis='y', linestyle="--", alpha=0.5)
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()


Casos Confirmados Ajustados por Población

In [ ]:


# Asegurarse de que las columnas estén en formato numérico
df_final["cumulative_confirmed"] = pd.to_numeric(df_final["cumulative_confirmed"], errors="coerce")
df_final["population"] = pd.to_numeric(df_final["population"], errors="coerce")

# Calcular casos confirmados por millón de habitantes
df_final["confirmed_per_million"] = (df_final["cumulative_confirmed"] / df_final["population"]) * 1_000_000

# Agrupar por país y calcular el promedio (por si hay varias fechas)
df_grouped = df_final.groupby("country_name")["confirmed_per_million"].mean().reset_index()

# Crear gráfico
plt.figure(figsize=(10,5))
sns.barplot(data=df_grouped, x="country_name", y="confirmed_per_million", palette="viridis")

# Mejorar visualización
plt.xlabel("País", fontsize=13)
plt.ylabel("Casos confirmados por millón de habitantes", fontsize=13)
plt.title("Casos Confirmados Ajustados por Población", fontsize=15, fontweight="bold")

# Cambiar formato del eje Y para mostrar números con separador de miles
plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f"{int(x):,}"))

plt.xticks(rotation=45)
plt.grid(axis='y', linestyle="--", alpha=0.5)
plt.tight_layout()
plt.show()




Vacunación Ajustada por Población

In [ ]:
import matplotlib.ticker as ticker

# Calcular dosis administradas por millón de habitantes con población única
df_grouped = df_final.groupby("country_name").agg({
    "cumulative_vaccine_doses_administered": "sum",
    "population": "first"  # O puedes usar 'max()' si prefieres asegurarlo
})

df_grouped["doses_per_million"] = (df_grouped["cumulative_vaccine_doses_administered"] / df_grouped["population"]) * 1_000_000
df_grouped = df_grouped.reset_index()

# Crear gráfico de barras corregido
plt.figure(figsize=(10,5))
sns.barplot(data=df_grouped, x="country_name", y="doses_per_million", palette="viridis")

# Mejorar visualización
plt.xlabel("País", fontsize=13)
plt.ylabel("Dosis administradas por millón de habitantes", fontsize=13)
plt.title("Vacunación Ajustada por Población", fontsize=15, fontweight="bold")

# Cambiar formato de números en el eje Y para evitar notación científica
plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f"{int(x):,}"))

plt.grid(axis='y', linestyle="--", alpha=0.5)
plt.show()


Impacto de Comorbilidades en la Mortalidad - Diabetes y Tabaquismo

In [ ]:
df_final["comorbidity_deaths_per_million"] = (df_final["comorbidity_mortality_rate"] * df_final["population"]) / 1_000_000

plt.figure(figsize=(10,5))
sns.barplot(data=df_final, x="country_name", y="comorbidity_deaths_per_million", palette="viridis")

plt.xlabel("País", fontsize=13)
plt.ylabel("Muertes por comorbilidades (por millón)", fontsize=13)
plt.title("Impacto de Comorbilidades en la Mortalidad", fontsize=15, fontweight="bold")
plt.grid(axis='y', linestyle="--", alpha=0.5)
plt.show()


Influencia de la Temperatura en la Propagación del COVID-19

In [ ]:


# Crear la métrica de casos confirmados por millón
df_final["confirmed_per_million"] = (df_final["cumulative_confirmed"] / df_final["population"]) * 1_000_000

# Crear gráfico de dispersión
plt.figure(figsize=(12,6))
ax = sns.scatterplot(data=df_final, x="average_temperature_celsius", y="confirmed_per_million", hue="country_name", palette="viridis", alpha=0.7, s=100)

# Agregar línea de tendencia
sns.regplot(data=df_final, x="average_temperature_celsius", y="confirmed_per_million", scatter=False, color="black", line_kws={"linewidth": 2, "linestyle": "--"})

# Mejorar visualización
plt.xlabel("Temperatura Media (°C)", fontsize=13)
plt.ylabel("Casos Confirmados por Millón de Habitantes", fontsize=13)
plt.title("Influencia de la Temperatura en la Propagación del COVID-19", fontsize=15, fontweight="bold")
plt.grid(True, linestyle="--", alpha=0.3)

plt.show()


Evolución de la Vacunación por País (2021 vs 2022)

In [ ]:

# Asegurar formato datetime
df_final["date"] = pd.to_datetime(df_final["date"])

# Calcular dosis por millón de habitantes
df_final["doses_per_million"] = (df_final["cumulative_vaccine_doses_administered"] / df_final["population"]) * 1_000_000

# Dividir por año
df_2021 = df_final[df_final["date"].dt.year == 2021]
df_2022 = df_final[df_final["date"].dt.year == 2022]

# Crear figura con subplots
fig, axes = plt.subplots(1, 2, figsize=(18, 6), sharey=True)

# Función para graficar en subplot
def graficar_sub(ax, df, año):
    sns.lineplot(
        data=df,
        x="date",
        y="doses_per_million",
        hue="country_name",
        palette="viridis",
        linewidth=2,
        ax=ax
    )
    ax.set_title(f"Vacunación en {año}", fontsize=14, fontweight='bold')
    ax.set_xlabel("Fecha", fontsize=12)
    ax.set_ylabel("Dosis por millón", fontsize=12 if ax == axes[0] else 0)
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
    ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
    ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'{int(x):,}'.replace(",", ".")))
    ax.tick_params(axis='x', rotation=45)
    ax.grid(True, linestyle="--", alpha=0.3)

# Graficar ambos años
graficar_sub(axes[0], df_2021, 2021)
graficar_sub(axes[1], df_2022, 2022)

# Ajustar diseño
plt.suptitle("Evolución de la Vacunación por País (2021 vs 2022)", fontsize=16, fontweight="bold")
plt.tight_layout(rect=[0, 0, 0.95, 0.95])
plt.show()



### AVANCE 3 - PROYECTO INTEGRADOR 

In [ ]:
#Importar Librerias
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

Analisis Exploratorio 

In [ ]:
#ruta al archivo csv
archivo = 'DatosFinalesFiltrado.csv'
df_final = pd.read_csv(archivo)
print(df_final.columns)


In [ ]:
# Convertir la columna de fecha en un formato datetime 
df_final["date"] = pd.to_datetime(df_final["date"])

# Asegurar que el DataFrame esté ordenado por fecha
df_final = df_final.sort_values(by="date")

# Verificar valores faltantes
print(df_final.isnull().sum())

# Resumen estadístico
print(df_final.describe())

In [ ]:
print(df_final.columns)


Evolución de Casos Activos vs. Recuperados 

In [ ]:

# Calcular recuperados y activos
df_final["cumulative_recovered"] = (df_final["cumulative_confirmed"] - df_final["cumulative_deceased"]).fillna(0)
df_final["active_cases"] = (df_final["cumulative_confirmed"] - df_final["cumulative_deceased"] - df_final["cumulative_recovered"]).clip(lower=0)

# Colores por país
colors = {
    "Argentina": "blue",
    "Peru": "lightblue",
    "Chile": "green",
    "Mexico": "orange",
    "Colombia": "red",
    "Brazil": "limegreen"
}

# Crear figura
plt.figure(figsize=(14,6))

# Graficar líneas
for country in df_final["country_name"].unique():
    subset = df_final[df_final["country_name"] == country]
    
    # Casos activos
    sns.lineplot(data=subset, x="date", y="active_cases", color=colors.get(country, "gray"), linewidth=2)
    
    # Casos recuperados
    sns.lineplot(data=subset, x="date", y="cumulative_recovered", color=colors.get(country, "gray"), linestyle="dashed", linewidth=2)

# Eje Y con formato
plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f"{int(x):,}"))

# Títulos y formato
plt.xlabel("Fecha", fontsize=13)
plt.xticks(rotation=45)
plt.ylabel("Número de casos", fontsize=13)
plt.title("Evolución de Casos Activos vs. Recuperados", fontsize=15, fontweight="bold")
plt.grid(True, linestyle="--", alpha=0.3)

# Crear leyendas por separado
activos_lines = [
    mlines.Line2D([], [], color=colors[c], linestyle="solid", linewidth=2, label=c)
    for c in colors.keys()
]

recuperados_lines = [
    mlines.Line2D([], [], color=colors[c], linestyle="dashed", linewidth=2, label=c)
    for c in colors.keys()
]

# Primera leyenda: Activos
first_legend = plt.legend(handles=activos_lines, title="Activos", loc="upper left", bbox_to_anchor=(1.01, 1))
plt.gca().add_artist(first_legend)

# Segunda leyenda: Recuperados
plt.legend(handles=recuperados_lines, title="Recuperados", loc="upper left", bbox_to_anchor=(1.01, 0.45))

plt.tight_layout()
plt.show()



Relación entre la Cobertura de Vacunación y la Reducción de Casos

In [ ]:


# Configurar estilo sin cuadriculado y con paleta Viridis
sns.set_style("white")
sns.set_palette("viridis")

# Asegurar que la columna 'date' está en formato datetime
df_final["date"] = pd.to_datetime(df_final["date"])

# Crear una nueva columna con el formato de mes-año
df_final["month"] = df_final["date"].dt.strftime("%b-%y")  # Ejemplo: 'Ene-21', 'Feb-21'

# Agrupar los datos por mes y sumar los valores
df_final_monthly = df_final.groupby("month").sum(numeric_only=True).reset_index()

# **Ordenar los meses en orden cronológico**
df_final_monthly["month_order"] = pd.to_datetime(df_final_monthly["month"], format="%b-%y")  # Convertir a datetime
df_final_monthly = df_final_monthly.sort_values("month_order")  # Ordenar por fecha

# Calcular el cambio mensual de casos confirmados
df_final_monthly["monthly_case_change"] = df_final_monthly["new_confirmed"].diff()

# Normalizar los valores para evitar diferencias de escala en el gráfico
df_final_monthly["normalized_vaccine_doses"] = df_final_monthly["cumulative_vaccine_doses_administered"] / df_final_monthly["cumulative_vaccine_doses_administered"].max()
df_final_monthly["normalized_case_change"] = df_final_monthly["monthly_case_change"] / df_final_monthly["monthly_case_change"].max()

# Aplicar una media móvil para suavizar la curva de casos confirmados
df_final_monthly["trend_case_change"] = df_final_monthly["normalized_case_change"].rolling(window=3).mean()

# Gráfico mejorado con paleta Viridis
plt.figure(figsize=(12, 6))

# Línea de dosis administradas (en escala normalizada)
sns.lineplot(x=df_final_monthly["month"], y=df_final_monthly["normalized_vaccine_doses"], label="Dosis Administradas", palette="viridis")

# Línea de reducción de casos confirmados (suavizada y normalizada)
sns.lineplot(x=df_final_monthly["month"], y=df_final_monthly["trend_case_change"], label="Reducción de Casos (Tendencia)", palette="viridis", linestyle="dashed")

# Ajustes finales
plt.title("Impacto de la Vacunación en la Reducción de Casos")
plt.xlabel("Mes-Año")
plt.ylabel("Proporción respecto al Máximo")
plt.xticks(rotation=45)  # Mejor legibilidad
plt.legend()
plt.show()


Distribución de la Población por Edad

In [ ]:


# Convertir la columna 'date' a string para evitar errores con timestamps
df_final["date"] = df_final["date"].astype(str)

# Transformar el DataFrame para formato adecuado de visualización
df_melted = df_final.melt(id_vars=["country_name"], 
                          value_vars=["population_age_00_09", "population_age_10_19", "population_age_20_29",
                                      "population_age_30_39", "population_age_40_49", "population_age_50_59",
                                      "population_age_60_69", "population_age_70_79", "population_age_80_and_older"],
                          var_name="Grupo de Edad", 
                          value_name="Porcentaje")

# Configurar estilo y paleta Viridis
sns.set_style("whitegrid")
sns.set_palette("viridis")

# Crear el gráfico de barras
plt.figure(figsize=(12, 6))
sns.barplot(x="country_name", y="Porcentaje", hue="Grupo de Edad", data=df_melted)

# Ajustes finales
plt.title("Distribución de la Población por Edad")
plt.xlabel("País")
plt.ylabel("Porcentaje de la Población")
plt.xticks(rotation=45)
plt.legend(title="Grupo de Edad")
plt.show()


Matriz de Correlación (Variables Seleccionadas)

In [ ]:

# Filtrar solo columnas clave para análisis
variables_relevantes = ["population_density", "life_expectancy", "pollution_mortality_rate", "adult_male_mortality_rate", "adult_female_mortality_rate"]
df_corr = df_final[variables_relevantes].corr()

# Configurar tamaño del gráfico
plt.figure(figsize=(8, 6))

# Generar el heatmap con la matriz de correlación
sns.heatmap(df_corr, annot=True, cmap="viridis", fmt=".2f", linewidths=0.5)

# Ajustes finales
plt.title("Matriz de Correlación (Variables Seleccionadas)")
plt.show()
